In [1]:
import pandas as pd
import os
import random
import time
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
data_dir = '/opt/ml/input/data/'
csv_file_path = os.path.join(data_dir, 'train_data_test_fe.csv')
df = pd.read_csv(csv_file_path)
df

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,year,month,day,hour,...,t_type_ans_mean,t_type_ans_sum,t_type_ans_std,t_type_time_mean,t_type_time_std,test_lvl,test_lvl_cat,test_type_lvl,test_type_lvl_cat,test_total_answer
0,0,A060001001,A060000001,1,1585009031,7224,2020,3,24,0,...,0.709232,187545,0.454118,55.175012,70.312176,23.794164,0,77.795457,1,0
1,0,A060001002,A060000001,1,1585009034,7225,2020,3,24,0,...,0.709232,187545,0.454118,55.175012,70.312176,23.794164,0,77.795457,1,1
2,0,A060001003,A060000001,1,1585009042,7225,2020,3,24,0,...,0.709232,187545,0.454118,55.175012,70.312176,23.794164,0,77.795457,1,2
3,0,A060001004,A060000001,1,1585009049,7225,2020,3,24,0,...,0.709232,187545,0.454118,55.175012,70.312176,23.794164,0,77.795457,1,3
4,0,A060001005,A060000001,1,1585009056,7225,2020,3,24,0,...,0.709232,187545,0.454118,55.175012,70.312176,23.794164,0,77.795457,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,1591339821,438,2020,6,5,6,...,0.702238,192246,0.457275,52.177665,66.330204,76.534202,4,74.301998,0,1389
2266582,7441,A040165001,A040000165,1,1597971999,8836,2020,8,21,1,...,0.684056,182864,0.464891,52.161823,66.587362,71.600279,4,76.253691,1,1092
2266583,7441,A040165002,A040000165,1,1597972010,8836,2020,8,21,1,...,0.684056,182864,0.464891,52.161823,66.587362,71.600279,4,76.253691,1,1093
2266584,7441,A040165003,A040000165,1,1597972056,8836,2020,8,21,1,...,0.684056,182864,0.464891,52.161823,66.587362,71.600279,4,76.253691,1,1094


In [3]:
def tag_fe(df):
    ## tag의 평균 정답률, 정답 총합, 표준편차
    correct_k = df.groupby(['KnowledgeTag'])['answerCode'].agg(['mean', 'sum','std'])
    correct_k.columns = ["tag_ans_mean", 'tag_ans_sum','tag_ans_std']
    df = pd.merge(df, correct_k, on=['KnowledgeTag'], how="left")
    
    ## 태그의 평균 풀이 시간, 표준편차
    time_k = df.groupby(['KnowledgeTag'])['time'].agg(['mean', 'std'])
    time_k.columns = ["tag_time_mean", 'tag_time_std']
    df = pd.merge(df, time_k, on=['KnowledgeTag'], how="left")

    ## 태그 난이도
    df['tag_lvl'] = df['tag_time_mean'] / df['tag_ans_mean']
    # 범주화
    tag_cat_num = 10
    df['tag_lvl_cat'] = pd.qcut(df['tag_lvl'],tag_cat_num,labels=[i for i in range(tag_cat_num)])

    ## 태그 노출 횟수
    df['tag_total_answer'] = df.groupby('KnowledgeTag')['answerCode'].cumcount()
    return df

In [4]:
df = tag_fe(df)

In [5]:
df
df.to_csv('/opt/ml/input/data/train_data_tag_fe.csv',index=False)